<a href="https://colab.research.google.com/github/ayulockin/debugNNwithWandB/blob/master/Pytorch_wandb_Exploding_Gradient_NaN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports and Setups

In [1]:
!pip install wandb -q

     |████████████████████████████████| 1.4MB 3.0MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 92kB 11.3MB/s 
     |████████████████████████████████| 460kB 58.7MB/s 
     |████████████████████████████████| 102kB 12.6MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 


In [0]:
import wandb

In [3]:
!wandb login

wandb: You can find your API key in your browser here: https://app.wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 69f60a7711ce6b8bbae91ac6d15e45d6b1f1430e
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


In [0]:
from sklearn.datasets import make_regression
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

## Prepare Data

In [0]:
# generate regression dataset
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=1)
# split into train and test
n_train = 500
x_train, x_test = X[:n_train, :], X[n_train:, :]
y_train, y_test = y[:n_train], y[n_train:]

y_train = y_train.reshape((y_train.shape+(1,)))
y_test = y_train.reshape((y_test.shape+(1,)))

In [0]:
x_train, y_train = torch.Tensor(x_train), torch.Tensor(y_train)
x_test, y_test = torch.Tensor(x_test), torch.Tensor(y_test)

trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train),
                                          batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test),
                                         batch_size=32, shuffle=True)

## Model

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(20, 25, bias=False)
        self.dropout = nn.Dropout(0.7)
        self.fc2 = nn.Linear(25, 1, bias=False)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x) ## Notice
        x = self.fc2(x)
        return x

In [0]:
def train(model, train_loader, optimizer, criterion, epoch, steps_per_epoch=20):
  # Switch model to training mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.train()
  train_total = 0
  train_correct = 0

  # We loop over the data iterator, and feed the inputs to the network and adjust the weights.
  for batch_idx, (data, target) in enumerate(train_loader, start=0):
    if batch_idx > steps_per_epoch:
      break
    # Load the input features and labels from the training dataset
    data, target = data, target
    
    # Reset the gradients to 0 for all learnable weight parameters
    optimizer.zero_grad()
    
    # Forward pass: Pass image data from training dataset, make predictions about class image belongs to (0-9 in this case)
    output = model(data)
    
    # Define our loss function, and compute the loss
    loss = criterion(output, target)
            
    # Backward pass: compute the gradients of the loss w.r.t. the model's parameters
    loss.backward()

    # Update the neural network weights
    optimizer.step()

  print('Epoch [{}], Loss: {} '.format(epoch, loss.item()), end='')
  wandb.log({'Train Loss': loss.item()})

In [0]:
def test(model, test_loader, criterion):
  # Switch model to evaluation mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.eval()
  
  test_loss = 0
  test_total = 0

  with torch.no_grad():
      for data, target in test_loader:
          # Load the input features and labels from the test dataset
          data, target = data, target
          
          # Make predictions: Pass image data from test dataset, make predictions about class image belongs to (0-9 in this case)
          output = model(data)
          
          # Compute the loss sum up batch loss
          test_loss += criterion(output, target).item()
          test_total += target.size(0)
          
  print(' Test_loss: {}'.format(test_loss/test_total))
  wandb.log({'Test Loss': test_loss/test_total})

In [10]:
net = Net()
print(net)

criterion = torch.nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

Net(
  (fc1): Linear(in_features=20, out_features=25, bias=False)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc2): Linear(in_features=25, out_features=1, bias=False)
)


In [11]:
wandb.init(project='explodingdebug')
wandb.watch(net, log='all')

for epoch in range(100):
  train(net, trainloader, optimizer, criterion, epoch)
  test(net, testloader, criterion)

print('Finished Training')

Epoch [0], Loss: nan  Test_loss: nan
Epoch [1], Loss: nan  Test_loss: nan
Epoch [2], Loss: nan  Test_loss: nan
Epoch [3], Loss: nan  Test_loss: nan
Epoch [4], Loss: nan  Test_loss: nan
Epoch [5], Loss: nan  Test_loss: nan
Epoch [6], Loss: nan  Test_loss: nan
Epoch [7], Loss: nan  Test_loss: nan
Epoch [8], Loss: nan  Test_loss: nan
Epoch [9], Loss: nan  Test_loss: nan
Epoch [10], Loss: nan  Test_loss: nan
Epoch [11], Loss: nan  Test_loss: nan
Epoch [12], Loss: nan  Test_loss: nan
Epoch [13], Loss: nan  Test_loss: nan
Epoch [14], Loss: nan  Test_loss: nan
Epoch [15], Loss: nan  Test_loss: nan
Epoch [16], Loss: nan  Test_loss: nan
Epoch [17], Loss: nan  Test_loss: nan
Epoch [18], Loss: nan  Test_loss: nan
Epoch [19], Loss: nan  Test_loss: nan
Epoch [20], Loss: nan  Test_loss: nan
Epoch [21], Loss: nan  Test_loss: nan
Epoch [22], Loss: nan  Test_loss: nan
Epoch [23], Loss: nan  Test_loss: nan
Epoch [24], Loss: nan  Test_loss: nan
Epoch [25], Loss: nan  Test_loss: nan
Epoch [26], Loss: nan 

In [0]:
def trainModified(model, train_loader, optimizer, criterion, epoch, steps_per_epoch=20):
  # Switch model to training mode. This is necessary for layers like dropout, batchnorm etc which behave differently in training and evaluation mode
  model.train()
  train_total = 0
  train_correct = 0

  # We loop over the data iterator, and feed the inputs to the network and adjust the weights.
  for batch_idx, (data, target) in enumerate(train_loader, start=0):
    if batch_idx > steps_per_epoch:
      break
    # Load the input features and labels from the training dataset
    data, target = data, target
    
    # Reset the gradients to 0 for all learnable weight parameters
    optimizer.zero_grad()
    
    # Forward pass: Pass image data from training dataset, make predictions about class image belongs to (0-9 in this case)
    output = model(data)
    
    # Define our loss function, and compute the loss
    loss = criterion(output, target)

    # Backward pass: compute the gradients of the loss w.r.t. the model's parameters
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 4.0)
    # Update the neural network weights
    optimizer.step()

  print('Epoch [{}], Loss: {} '.format(epoch, loss.item()), end='')
  wandb.log({'Train Loss': loss.item()})


In [13]:
del net
net = Net()
print(net)

criterion = torch.nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, dampening=1e-6)

Net(
  (fc1): Linear(in_features=20, out_features=25, bias=False)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc2): Linear(in_features=25, out_features=1, bias=False)
)


In [14]:
wandb.init(project='explodingdebug')
wandb.watch(net, log='all')

for epoch in range(100):
  trainModified(net, trainloader, optimizer, criterion, epoch)
  test(net, testloader, criterion)

print('Finished Training')

Epoch [0], Loss: 24005.736328125  Test_loss: 648.1969921875
Epoch [1], Loss: 13096.1015625  Test_loss: 656.34944921875
Epoch [2], Loss: 11086.3447265625  Test_loss: 667.07905078125
Epoch [3], Loss: 6045.63427734375  Test_loss: 758.681412109375
Epoch [4], Loss: 1780.2254638671875  Test_loss: 1005.92348828125
Epoch [5], Loss: 427.66156005859375  Test_loss: 1372.0641875
Epoch [6], Loss: 26.74477195739746  Test_loss: 1371.44294140625
Epoch [7], Loss: 23.26637840270996  Test_loss: 1382.2956015625
Epoch [8], Loss: 11.095369338989258  Test_loss: 1403.86296484375
Epoch [9], Loss: 5.545734405517578  Test_loss: 1408.36526171875
Epoch [10], Loss: 2.4922313690185547  Test_loss: 1388.17856640625
Epoch [11], Loss: 4.9311065673828125  Test_loss: 1390.03403515625
Epoch [12], Loss: 1.9409598112106323  Test_loss: 1390.3574296875
Epoch [13], Loss: 1.8449726104736328  Test_loss: 1392.67828125
Epoch [14], Loss: 2.713657855987549  Test_loss: 1378.0353125
Epoch [15], Loss: 2.134899616241455  Test_loss: 1395.